In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

## Compute the number of restaurants per each cuisine using the provided nyc_restaurant_inspections.csv data set. 
#### - The cuisine type can be extracted from the "CUISINE DESCRIPTION" column. Note that, a single restaurant may be inspected multiple times. 
#### - You must use the "CAMIS" column to keep only unique restaurants while computing the number of restaurants per cuisine.

In [3]:
cuisine = sc.textFile('nyc_restaurant_inspections.csv', use_unicode=False).cache()

In [4]:
cuisine.first()

'CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE'

In [5]:
list(enumerate(cuisine.first().split(',')))

[(0, 'CAMIS'),
 (1, 'DBA'),
 (2, 'BORO'),
 (3, 'BUILDING'),
 (4, 'STREET'),
 (5, 'ZIPCODE'),
 (6, 'PHONE'),
 (7, 'CUISINE DESCRIPTION'),
 (8, 'INSPECTION DATE'),
 (9, 'ACTION'),
 (10, 'VIOLATION CODE'),
 (11, 'VIOLATION DESCRIPTION'),
 (12, 'CRITICAL FLAG'),
 (13, 'SCORE'),
 (14, 'GRADE'),
 (15, 'GRADE DATE'),
 (16, 'RECORD DATE'),
 (17, 'INSPECTION TYPE')]

In [6]:
cuisine.count()

379675

In [7]:
def parseRows(i, rows):
    if i == 0: 
        rows.next() # next(row)
    import csv
    rows = csv.reader(rows)
    for x in rows:
        yield x[0], x[7]

In [10]:
r = (cuisine.mapPartitionsWithIndex(parseRows)
            .reduceByKey(lambda accum, y: y)
            .map(lambda (camis, cuisine): (cuisine, 1))
            .reduceByKey(lambda accum, n: accum + n)
            .sortBy(lambda x: -x[1]))

In [11]:
r.take(30)

[('American', 6002),
 ('Chinese', 2399),
 ('Caf\xc3\x83\xc2\xa9/Coffee/Tea', 1629),
 ('Other', 1296),
 ('Pizza', 1186),
 ('Italian', 1016),
 ('Mexican', 877),
 ('Japanese', 859),
 ('Latin (Cuban, Dominican, Puerto Rican, South & Central American)', 840),
 ('Bakery', 733),
 ('Caribbean', 671),
 ('Spanish', 644),
 ('Donuts', 537),
 ('Pizza/Italian', 483),
 ('Chicken', 456),
 ('Sandwiches', 406),
 ('Juice, Smoothies, Fruit Salads', 382),
 ('Hamburgers', 378),
 ('Asian', 371),
 ('Ice Cream, Gelato, Yogurt, Ices', 339),
 ('Indian', 332),
 ('Jewish/Kosher', 327),
 ('French', 319),
 ('Delicatessen', 294),
 ('Thai', 286),
 ('Korean', 276),
 ('Sandwiches/Salads/Mixed Buffet', 253),
 ('Mediterranean', 253),
 ('Irish', 187),
 ('Seafood', 181)]